In [1]:
import mlflow
import dagshub
dagshub.init(repo_owner='satyajeetrai007', repo_name='Youtube-Comment-Sentiment-Analysis', mlflow=True)

Accessing as satyajeetrai007

Initialized MLflow to track repo "satyajeetrai007/Youtube-Comment-Sentiment-Analysis"

Repository satyajeetrai007/Youtube-Comment-Sentiment-Analysis initialized!

In [2]:
mlflow.set_experiment("TfIdf Trigram max_features")

2025/08/25 19:20:28 INFO mlflow.tracking.fluent: Experiment with name 'TfIdf Trigram max_features' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/665f040d82164edc89eb8af6b61adea1', creation_time=1756129829833, experiment_id='3', last_update_time=1756129829833, lifecycle_stage='active', name='TfIdf Trigram max_features', tags={}>

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle
import os

In [4]:
df = pd.read_csv('data_preprocessed.csv').dropna(subset=['clean_comment']).drop_duplicates()
df.shape

(36243, 2)

In [13]:
# Step 1: Function to run the experiment
def run_experiment_tfidf_max_features(max_features):
    ngram_range = (1, 3)  # Trigram setting

    # Step 2: Vectorization using TF-IDF with varying max_features
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    # Step 4: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"TFIDF_Trigrams_max_features_{max_features}")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with TF-IDF Trigrams, max_features={max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Trigrams, max_features={max_features}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # Log the model
        with open( f"random_forest_model_tfidf_trigrams_{max_features}", "wb") as f:
            pickle.dump(model, f)

        # Load back (CI/CD reproducibility)
        with open(f"random_forest_model_tfidf_trigrams_{max_features}", "rb") as f:
            clf = pickle.load(f)

  
        mlflow.log_artifact(f"random_forest_model_tfidf_trigrams_{max_features}", "model")

In [14]:

# Step 6: Test various max_features values
max_features_values = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for max_features in max_features_values:
    run_experiment_tfidf_max_features(max_features)

🏃 View run TFIDF_Trigrams_max_features_1000 at: https://dagshub.com/satyajeetrai007/Youtube-Comment-Sentiment-Analysis.mlflow/#/experiments/3/runs/6aa689bae26145d29829edb555d8b66b
🧪 View experiment at: https://dagshub.com/satyajeetrai007/Youtube-Comment-Sentiment-Analysis.mlflow/#/experiments/3
🏃 View run TFIDF_Trigrams_max_features_2000 at: https://dagshub.com/satyajeetrai007/Youtube-Comment-Sentiment-Analysis.mlflow/#/experiments/3/runs/a38be86c4e31459684f8f4dcca4760d7
🧪 View experiment at: https://dagshub.com/satyajeetrai007/Youtube-Comment-Sentiment-Analysis.mlflow/#/experiments/3
🏃 View run TFIDF_Trigrams_max_features_3000 at: https://dagshub.com/satyajeetrai007/Youtube-Comment-Sentiment-Analysis.mlflow/#/experiments/3/runs/1055d70a0f1142b0b54072e18f90c0b0
🧪 View experiment at: https://dagshub.com/satyajeetrai007/Youtube-Comment-Sentiment-Analysis.mlflow/#/experiments/3
🏃 View run TFIDF_Trigrams_max_features_4000 at: https://dagshub.com/satyajeetrai007/Youtube-Comment-Sentiment-An

In [ ]:
run_experiment_tfidf_max_features(1500)